# 时序异常检测基准测试框架

这个notebook是整个基准测试框架的核心运行脚本，用于:
- 加载和管理实验配置
- 数据集处理（未完成）
- 执行完整的模型训练和评估流程
- 生成结果报告和可视化（待添加新评估图像和指标）


In [1]:
# 导入依赖库
import os
import sys
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
# warnings.filterwarnings('ignore')





In [2]:
# # 放在任何 torch 操作之前（越靠前越好）
# import os

# # 1) 让 MIOpen 避免在第一次做耗时的穷举搜索（显著减少“第一次超久”）
# os.environ['MIOPEN_FIND_ENFORCE'] = '1'  # 只用已知解，跳过搜索
# # 可选：自定义/确保可写缓存目录，避免权限问题导致每次都重新搜索
# os.environ['MIOPEN_CACHE_DIR'] = os.path.expanduser('~/.cache/miopen')

# # 2) 可选：看日志定位问题时再启用
# # os.environ['MIOPEN_LOG_LEVEL'] = '5'

# # 3) 限定可见 GPU（如有多卡）
# # os.environ['HIP_VISIBLE_DEVICES'] = '0'

# # 4) 小型 warmup，提前完成 HIP/MIOpen 初始化（放在创建模型前）
# import torch
# if torch.cuda.is_available():
#     x = torch.randn(128, 128, device='cuda')
#     y = x @ x.t()
#     torch.cuda.synchronize()

In [3]:
# 添加项目路径
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"项目根目录: {project_root}")
print("依赖库导入完成")

# 分步导入框架组件
try:
    from src.config_manager import ConfigManager
    print("ConfigManager 导入成功")
except ImportError as e:
    print(f"ConfigManager 导入失败: {e}")

try:
    from src.dataloaders import DataPipeline
    print("DataPipeline 导入成功")
except ImportError as e:
    print(f"DataPipeline 导入失败: {e}")

try:
    from models.base_model import ModelFactory
    print("ModelFactory 导入成功")
except ImportError as e:
    print(f"ModelFactory 导入失败: {e}")

try:
    from src.trainer import Trainer
    print("Trainer 导入成功")
except ImportError as e:
    print(f"Trainer 导入失败: {e}")

try:
    from src.metrics import TimeSeriesEvaluator, print_evaluation_report
    print("Metrics 导入成功")
except ImportError as e:
    print(f"Metrics 导入失败: {e}")

# 新增：模型加载器与模块化评估器
try:
    from src.model_loader import ModelLoader
    print("ModelLoader 导入成功")
except ImportError as e:
    print(f"ModelLoader 导入失败: {e}")

try:
    from src.eval_plugins import ModularEvaluator, MetricRegistry
    print("ModularEvaluator/MetricRegistry 导入成功")
except ImportError as e:
    print(f"Eval plugins 导入失败: {e}")

# 设置随机种子
import random
random.seed(42)
np.random.seed(42)

print("导入检测完成")

项目根目录: /home/chen/dev/fault_diagnosis_lib/benchmark
依赖库导入完成
ConfigManager 导入成功
DataPipeline 导入成功
ModelFactory 导入失败: No module named 'models.base_model'
Trainer 导入失败: No module named 'src.trainer'
Metrics 导入失败: No module named 'src.metrics'
ModelLoader 导入失败: No module named 'src.model_loader'
Eval plugins 导入失败: No module named 'src.eval_plugins'
导入检测完成


## 1. 配置管理与实验设置

In [4]:
# 加载用户的实验配置
config_file = project_root / 'configs' / 'default_experiment.yaml'
# config_file = "/home/chen/dev/fault_diagnosis_lib/benchmark/configs/default_experiment.yaml"
config_manager = ConfigManager(config_dir = config_file)
config = config_manager.load_config()

print("实验配置:")
print(f"   - 实验名称: {config['experiment']['name']}")
print(f"   - 数据集: {config['data']['dataset']}")
print(f"   - 模型: {list(config['models'].keys())}")
print(f"   - 评估指标: {config['evaluation']['metrics']}")
print(f"   - 结果保存: {config['output']['save_results']}")

实验配置:
   - 实验名称: benchmark_experiment_01
   - 数据集: NPY_UCI_HAR
   - 模型: ['lstm']
   - 评估指标: ['f1', 'precision', 'recall', 'auc', 'f1_point_adjusted']
   - 结果保存: True


## 2. 数据加载与预处理

In [5]:
# 创建数据管道
data_pipeline = DataPipeline(config)

# 加载数据
dataset_name = config['data']['dataset']
print(f"加载数据集: {dataset_name}")

# get train_data, train_label, test_data, test_label, metadata
train_data, test_data, train_label, test_label, metadata = data_pipeline.prepare_data()

加载数据集: NPY_UCI_HAR
Project root path: /home/chen/dev/fault_diagnosis_lib/benchmark
Loading file: /home/chen/dev/fault_diagnosis_lib/benchmark/data/NPY_UCI_HAR/train_X.npy
Project root path: /home/chen/dev/fault_diagnosis_lib/benchmark
Loading file: /home/chen/dev/fault_diagnosis_lib/benchmark/data/NPY_UCI_HAR/train_y.npy
Project root path: /home/chen/dev/fault_diagnosis_lib/benchmark
Loading file: /home/chen/dev/fault_diagnosis_lib/benchmark/data/NPY_UCI_HAR/test_X.npy
Project root path: /home/chen/dev/fault_diagnosis_lib/benchmark
Loading file: /home/chen/dev/fault_diagnosis_lib/benchmark/data/NPY_UCI_HAR/test_y.npy
Metadata: DataMetadata(dataset_name='NPY_UCI_HAR', fault_type='multi-class', feature_dim=9, num_classes=6, timesteps=128, number_train_samples=7352, number_test_samples=2947)
No normalization applied.


In [6]:
# show shapes
print(f"训练数据形状: {train_data.shape}, 训练标签形状: {train_label.shape}")
print(f"测试数据形状: {test_data.shape}, 测试标签形状: {test_label.shape}")
print(f"数据元信息: {metadata}")

训练数据形状: (7352, 128, 9), 训练标签形状: (7352,)
测试数据形状: (2947, 128, 9), 测试标签形状: (2947,)
数据元信息: DataMetadata(dataset_name='NPY_UCI_HAR', fault_type='multi-class', feature_dim=9, num_classes=6, timesteps=128, number_train_samples=7352, number_test_samples=2947)


## 4. 结果对比与可视化